In [ ]:
%matplotlib inline      
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import math
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.api import OLS, add_constant, graphics
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import norm,spearmanr,describe
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LogisticRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_validate, KFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,ConfusionMatrixDisplay,precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from patsy import dmatrices
import time
import backtrader as bt
from statistics import mean
sns.set_style('whitegrid')   
import quantstats
quantstats.extend_pandas()

## Backtest

In [ ]:
def preprocesamiento(dataframe_datos):
    dataframe_datos_procesados=dataframe_datos
    return(dataframe_datos_procesados)

def regla_trading(dataframe_precio_actual,dataframe_datos,comision,dataframe_trades_anteriores,fecha_actual):
    cantidad=(dataframe_datos.loc[str(fecha_actual)]*2-1)

    #if dataframe_datos.loc[str(fecha_actual)]>=0.8:
     #   cantidad=0.1
    #if dataframe_datos.loc[str(fecha_actual)]<=0.2:
     #   cantidad=-0.1
    #else:
    cantidad=1
    return(cantidad)   # Cantidad a comprar

    
class Estrategia:
    def __init__(self ,dataframe_precio):
        self.comision=0
        self.dataframe_precio=dataframe_precio
        
    def add_comision(self,comision):
        self.comision=comision
        
    def add_datos(self,dataframe_datos):
        self.dataframe_datos=preprocesamiento(dataframe_datos)
        
    def run(self):   #dataframe_datos tiene que ser serie de tiempo
        self.dataframe_trades_anteriores=pd.DataFrame()
        self.dataframe_cantidad_comprada=pd.DataFrame()
        self.dataframe_pnl=pd.DataFrame()
        self.dataframe_precio=self.dataframe_precio.iloc[-self.dataframe_datos.shape[0]:]
        for i in self.dataframe_precio.index:
            dataframe_precio_actual=self.dataframe_precio.loc[:i]    # ver los datos para forward looking # cortar dataframe_datos
            accion_actual=regla_trading(dataframe_precio_actual,self.dataframe_datos,self.comision,self.dataframe_trades_anteriores,i)
            self.dataframe_trades_anteriores=self.dataframe_trades_anteriores.append(pd.DataFrame([accion_actual],index=[i]))
            cantidad_comprada=accion_actual
            self.dataframe_cantidad_comprada=self.dataframe_cantidad_comprada.append(pd.DataFrame([cantidad_comprada],index=[i]))
        self.dataframe_cantidad_comprada.columns=["Cantidad Comprada"]
        self.dataframe_pnl=dataframe_precio_actual.merge(self.dataframe_cantidad_comprada, how='outer', left_index=True, right_index=True)
        self.dataframe_pnl["Cantidad Comprada acomulada"]=self.dataframe_pnl["Cantidad Comprada"].cumsum()
        self.dataframe_pnl["Comision"] = self.dataframe_pnl["Cantidad Comprada"].apply(lambda x: self.comision if(x!=0) else 0)
        self.dataframe_pnl=self.dataframe_pnl.assign(Dinero_usado=lambda x: x.iloc[:,0]*x["Cantidad Comprada"]+x["Comision"])
        self.dataframe_pnl["Dinero usado total"]=self.dataframe_pnl["Dinero_usado"].cumsum()
        self.dataframe_pnl["Pnl hasta la fecha"]=self.dataframe_pnl["Cantidad Comprada acomulada"]*self.dataframe_pnl.iloc[:,0]-self.dataframe_pnl["Dinero usado total"]
        self.dataframe_pnl_vector=self.dataframe_pnl["Pnl hasta la fecha"]
        self.dataframe_pnl_vector_diff=self.dataframe_pnl_vector.diff()
    def add_benchmark(self,benchmark):
        self.benchmark=benchmark

    def plot(self,dt):
        dataframe_pnl_dt=self.dataframe_dt(dt)
        self.stats(dataframe_pnl_dt)
        fig, ax =plt.subplots(1,1)  
        data=[[self.hit_ratio,self.statistics[2],(self.statistics[3])**(1/2),self.sharpe_ratio,self.statistics[4],self.statistics[5]]]
        column_labels=["Hit ratio","Mean", "Standard deviation","Sharpe Ratio","Skew","Kurtosis"]
        ax.axis('tight')
        ax.axis('off')
        table=ax.table(cellText=data,colLabels=column_labels,loc="center")
        table.scale(4, 6)
        table.set_fontsize(64)
        fig, ax =plt.subplots(1,1)   
        data=[[self.daily_max_drawdown,self.max_drawdown,self.var_5,self.var_1]]
        column_labels=["1 period Max Drawdown","Max Drawdown","VAR 5%","VAR 1%"]
        ax.axis('tight')
        ax.axis('off')
        table=ax.table(cellText=data,colLabels=column_labels,loc="center")
        table.scale(4, 6)
        table.set_fontsize(64)
        
        fig,(ax1) = plt.subplots(ncols=1, figsize=(12,6))
        ax1=sns.histplot(dataframe_pnl_dt,kde=True).set(title='Histograma Pnl')
        fig,(ax1) = plt.subplots(ncols=1, figsize=(12,6))
        ax1=plt.plot(dataframe_pnl_dt.cumsum().index,dataframe_pnl_dt.cumsum())
        plt.xticks(rotation=90)
        plt.title("Pnl Acomulado")
        fig,(ax1) = plt.subplots(ncols=1, figsize=(12,6))
        ax1=plt.plot(self.drawdown.index,self.drawdown)
        plt.xticks(rotation=90)
        plt.title("DrawDowns")
        
        fig,(ax1,ax2) = plt.subplots(ncols=2, figsize=(12,6))
        self.var_5_dataframe_20_periods.plot(ax=ax1,title="Var 5% 20 Periodos")
        ax1.text(x=.05, y=.9,s=f'Worst 5%: {self.var_5_dataframe_20_periods.quantile(0.05):5.2f}\n Worst 1%: {self.var_5_dataframe_20_periods.quantile(0.01):5.2f}',horizontalalignment='left',verticalalignment='center',transform=ax2.transAxes)
        
        self.var_5_dataframe_250_periods.plot(ax=ax2,title="Var 5% 250 Periodos")
        #ax1.text(x=.05, y=.9,s=f'Worst 5%: {self.var_5_dataframe_20_periods.quantile(0.05):5.2f}\n Worst 1%: {self.var_5_dataframe_20_periods.quantile(0.01):5.2f}',horizontalalignment='left',verticalalignment='center',transform=ax2.transAxes)
        ax2.text(x=.05, y=.9,s=f'Worst 5%: {self.var_5_dataframe_250_periods.quantile(0.05):5.2f}\n Worst 1%: {self.var_5_dataframe_250_periods.quantile(0.01):5.2f}',horizontalalignment='left',verticalalignment='center',transform=ax2.transAxes)
        plt.xticks(rotation=90)
        
        fig,(ax1,ax2) = plt.subplots(ncols=2, figsize=(12,6))
        self.var_1_dataframe_20_periods.plot(ax=ax1,title="Var 1% 20 Periodos")
        self.var_1_dataframe_250_periods.plot(ax=ax2,title="Var 1% 250 Periodos")
        plt.xticks(rotation=90)
        ax1.text(x=.05, y=.9,s=f'Worst 5%: {self.var_1_dataframe_20_periods.quantile(0.05):5.2f}\n Worst 1%: {self.var_1_dataframe_20_periods.quantile(0.01):5.2f}',horizontalalignment='left',verticalalignment='center',transform=ax2.transAxes)
        ax2.text(x=.05, y=.9,s=f'Worst 5%: {self.var_1_dataframe_250_periods.quantile(0.05):5.2f}\n Worst 1%: {self.var_1_dataframe_250_periods.quantile(0.01):5.2f}',horizontalalignment='left',verticalalignment='center',transform=ax2.transAxes)
        
        fig,(ax1) = plt.subplots(ncols=1, figsize=(12,6))
        self.desv_estandar_20_periodos.plot(ax=ax1,title="Desviación estándar 20 Periodos")
        plt.xticks(rotation=90)
        
        
    def dataframe_dt(self,dt):
        return(self.dataframe_pnl_vector.iloc[::dt].diff())    
    
    def stats(self,dataframe_pnl_vector_diff):
        self.sharpe_ratio=dataframe_pnl_vector_diff.mean()/dataframe_pnl_vector_diff.std()  # ver si usar log returns o diff en el pnl
        self.hit_ratio=[1 if i>0 else 0 for i in list(dataframe_pnl_vector_diff)]
        self.hit_ratio=mean(self.hit_ratio)
        self.statistics=describe(dataframe_pnl_vector_diff.dropna() , bias=False)
        self.var_5=dataframe_pnl_vector_diff.quantile(q=0.05)
        self.var_1=dataframe_pnl_vector_diff.quantile(q=0.01)
        self.var_5_dataframe_20_periods=dataframe_pnl_vector_diff.rolling(window=20).quantile(0.05)
        self.var_5_dataframe_250_periods=dataframe_pnl_vector_diff.rolling(window=250).quantile(0.05)
        self.var_1_dataframe_20_periods=dataframe_pnl_vector_diff.rolling(window=20).quantile(0.01)
        self.var_1_dataframe_250_periods=dataframe_pnl_vector_diff.rolling(window=250).quantile(0.01)
        self.drawdown=dataframe_pnl_vector_diff.apply(lambda x: x if x<=0 else np.nan).fillna(method="ffill").dropna()
        self.daily_max_drawdown=dataframe_pnl_vector_diff.cummin()[-1]
        
        roll_max=self.dataframe_pnl_vector_diff.rolling(window=250,min_periods=1).max()
        dd=estrategia_test.dataframe_pnl_vector_diff/roll_max-1.0
        max_drawdown=dd.rolling(window=250,min_periods=1).min()*roll_max
        self.max_drawdown=max_drawdown.iloc[-1]
        
        self.desv_estandar_20_periodos=self.dataframe_pnl_vector_diff.rolling(window=20).std()
        #returns=self.dataframe_pnl.iloc[:,0].pct_change()
        #self.precio_log_ret = np.log(1+returns) 

In [ ]:
estrategia_test=Estrategia(pd.DataFrame(df_new.loc[:,f'{target.split(" ")[0]} {target.split(" ")[1]}']))
estrategia_test.add_datos(df_fair_value_valor_actual.iloc[:,2])
estrategia_test.run()

In [ ]:
estrategia_test.plot(1)